## Basic Generation Training

In [ ]:
# Change to correct directory
%cd ../MedSinGAN/

In [ ]:
# Run a best combination for a given image
! python main_train.py --train_mode generation --input_name ../pipeline_data_tcia/normal/normal.png --train_stages 24 --niter 2500 --gpu 0 --train_depth 20 --im_max_size 614 --batch_norm --activation prelu --convergence_patience 2500

## Combination Generation Testing

In [ ]:
# Change to correct directory
%cd ../MedSinGAN/

In [ ]:
# Required imports
import sys
sys.path.append("../utils")
from utils import execute_bash_command

In [ ]:
# Parameters variation
# (Stages, N Iterations, Concurrent Stages, Activation Function)
params_variations = [(16,2500,12,"prelu"),(16,2500,14,"prelu"),(16,2500,16,"prelu"),
                     (24,2500,22,"prelu"),(24,2500,20,"prelu"),(30,2500,28,"prelu"),
                     (30,2500,24,"prelu")]

In [ ]:
params_variations = [(24,2500,20,"prelu")]

In [ ]:
# Execute single experiment with given parameters
def do_single_experiment(image_pth, stages, niter, concurrent, a_func):
    cmd = f"python main_train.py --train_mode generation --input_name {image_pth} --im_max_size 614 --train_stages {stages} --niter {niter} --gpu 0 --train_depth {concurrent} --activation {a_func} --batch_norm"
    for path in execute_bash_command(cmd.split()):
        print(path, end="")
        
    comand = f"zip -r ../(G)S{stages}_NITER{niter}_CONC{concurrent}_AFUNC{a_func}.zip ."
    for path in execute_bash_command(comand.split()):
        print(path, end="")
    
    ! rm -r mlruns TrainedModels

In [ ]:
# Run experimentations
for comb in params_variations:
    do_single_experiment('../experiments/harmonized_w_mask.jpg', comb[0], comb[1], comb[2], comb[3])

## Evaluation

In [ ]:
# Example of generation evaluation
from evaluate_generation import GenerationEvaluator

base_image = "target.png"
generated_samples = "folder_path"

evaluator = GenerationEvaluator(base_image, generated_samples, adjust_sizes=True)
print('FID:', evaluator.run_fid())
print('LPIPS:', evaluator.run_lpips())
ssim, ms_ssim = evaluator.run_mssim()
print('SSIM:', ssim)
print('MS-SSIM:', ms_ssim)

## Save Model

In [ ]:
# Zip everything from this model
! zip -r current_model.zip .

## Delete Current Model

In [ ]:
# Delete everything that the model produces
! rm -r mlruns TrainedModels

# Generator Fine-Tune

In [ ]:
# Run a best combination for a given image
! python main_train.py --train_mode generation --input_name ../pipeline_data_tcia/normal/normal.png --train_stages 24 --niter 2500 --gpu 0 --train_depth 20 --im_max_size 614 --batch_norm --activation prelu --convergence_patience 2500 --g_optimizer_folder

In [ ]:
! python evaluate_model.py --gpu 0 --num_samples 50 --model_dir